In [ ]:
from polars import DataFrame, col, when, lit
from collections import namedtuple
from polars import Int32


In [ ]:
Tx = namedtuple("Tx", ("ts", "price", "qty"))
txs = [
    Tx(1, 100, 10),
    Tx(2, 101, -2),
    Tx(3, 102, 2),
    Tx(4, 103, -10),
    Tx(5, 104, 5),
    Tx(6, 105, 5),
    Tx(7, 106, -10),
    Tx(8, 107, 3),
    Tx(9, 108, 3),
]

In [83]:



def calc_tx_pnl(txs: list[any])-> DataFrame:
    price = col("price")
    qty = col("qty")
    running_qty = col("running_qty")
    # running_cost = col("running_cost")
    stretch = col("stretch")
    running_avg_price = col("running_avg_price")

    running_buy_only_cost = when(qty>0).then(qty*price).otherwise(0).cum_sum().over(stretch)
    running_buy_only_qty = when(qty>0).then(qty).otherwise(0).cum_sum().over(stretch)


    df = DataFrame(txs)
    df = df.with_columns(running_qty=qty.cum_sum()).with_columns(
        stretch=(running_qty <= 0).shift(1).cast(Int32).cum_sum().fill_null(0),
        # sell_all=(running_qty == 0),
    ).with_columns(
        running_cost=when(running_qty <= 0)
        .then(lit(0))
        .otherwise((qty * price).cum_sum().over(stretch)),
    ).with_columns(running_avg_price=(running_buy_only_cost / running_buy_only_qty).fill_nan(0)).with_columns(
        realized_pnl=when(qty < 0)
        .then((price-running_avg_price)*-qty)
        .otherwise(None),
        cost=qty*running_avg_price
    )
    return df

pnl = calc_tx_pnl(txs)
pnl

ts,price,qty,running_qty,stretch,running_cost,running_avg_price,realized_pnl,cost
i64,i64,i64,i64,i32,i64,f64,f64,f64
1,100,10,10,0,1000,100.0,null,1000.0
2,101,-2,8,0,798,100.0,2.0,-200.0
3,102,2,10,0,1002,100.333333,null,200.666667
4,103,-10,0,0,0,100.333333,26.666667,-1003.333333
5,104,5,5,1,520,104.0,null,520.0
6,105,5,10,1,1045,104.5,null,522.5
7,106,-10,0,1,0,104.5,15.0,-1045.0
8,107,3,3,2,321,107.0,null,321.0
9,108,3,6,2,645,107.5,null,322.5


In [92]:
def realized_pnl(pnl:DataFrame)->tuple[float,float]:
  return pnl.select(
    col('running_cost').last(),
    col('realized_pnl').fill_null(0).sum()
  )

realized_pnl(pnl)

running_cost,realized_pnl
i64,f64
645,43.666667
